In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tf_retinanet
from tf_retinanet.builders import application_builder
from tf_retinanet.builders import loss_builder
from tf_retinanet.builders import sample_builder
from tqdm import tqdm

In [2]:
def build_dataset():
    labels = pd.read_csv(os.path.join('head', 'labels.csv'))
    annotations = pd.read_csv(os.path.join('head', 'boxes.csv'))
    annotations['classes'] = tf_retinanet.to_classes(annotations['name'].values)
    with tf_retinanet.data.Shards(os.path.join('data', 'dataset.record')) as shards:
        for index, label in tqdm(labels.iterrows()):
            path = os.path.join('head', 'images', label['filename'])
            image = tf_retinanet.image.read(path)
            _annotations = annotations[annotations['image_id'] == label['id']]
            bbox = _annotations[['xmin', 'ymin', 'xmax', 'ymax']]
            image, bbox = tf_retinanet.transform(image, bbox.values, min_side=200, max_side=300)

            # create batch, batch_size=1
            classes = np.zeros(shape=(1, _annotations['classes'].values.shape[0], 1))
            classes[0, :, 0] = _annotations['classes'].values
            bboxses = np.zeros(shape=(1,) + bbox.shape)
            bboxses[0, :] = bbox
            images = np.zeros(shape=(1,) + image.shape)
            images[0, :] = image

            sample = sample_builder.build(images, bboxses, classes, 1)
            shards.write(sample)

In [3]:
build_dataset()

2404it [02:40, 14.98it/s]


In [4]:
dataset = tf_retinanet.data.read(os.path.join('data', 'dataset.record-?????-of-00010'))

In [5]:
model = application_builder.build('mobilenet_v1', num_classes=1, num_anchors=9)

In [6]:
model = model
model.compile(optiomizer=tf.keras.optimizers.Adam(lr=1e-5),
             loss=[loss_builder.build_smooth_l1_loss(), loss_builder.build_focal_loss()])

In [7]:
def input_fn(dataset):
    while True:
        dataset = dataset.shuffle(1000)
        for image, bboxes, classes in dataset:
            yield (image, [bboxes, classes])

In [8]:
model.fit_generator(input_fn(dataset), steps_per_epoch=1000)

1000/1000 [==============================] - 575s 575ms/step - loss: 5.9981 - localization_loss: 2.6393 - classification_loss: 3.3588


In [15]:
from time import time
at = time()
for image, _, _ in dataset.take(10):
    st = time()
    bboxes, classes = model(image)
et = time()
print('performance for 10 examples:', et - at)
print('performance for last example:', et - st)

performance for 10 examples: 0.749924898147583
performance for last example: 0.062482357025146484
